In [33]:
import pandas as pd
df = pd.read_excel('data/Educação.xlsx')
resultado = df 
resultado.columns


Index(['Município', 'IDHM 2010 ', 'População (Censo 2022)', 'Território',
       'IDEB AI - Pública', 'IDEB Anos Finais - Pública',
       'SAEPI Português 2º ano Ensino Fundamental ',
       'SAEPI Matemática 2º ano Ensino Fundamental ',
       'Taxa de evasão ensino fund. Anos Iniciais',
       'Taxa de evasão ensino fund. Anos Finais ',
       'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais'],
      dtype='object')

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Assumindo que sua variável se chama 'resultado'
df = resultado.copy()

# Remover espaços extras nos nomes das colunas
df.columns = [col.rstrip() for col in df.columns]

# Função para tratar valores como "#VALUE!" e converter strings com vírgula para float
def tratar_valor(x):
    if isinstance(x, str):
        if x == "#VALUE!":
            return np.nan  # Usar NaN para depois preencher com a média
        # Substituir vírgula por ponto para números em formato brasileiro
        return float(x.replace(',', '.'))
    return x

# Definir os indicadores para análise (excluindo SAEPI, Território e População)
indicadores_para_analise = [
    'IDHM 2010',
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

# Converter valores para numérico
for col in indicadores_para_analise:
    if col in df.columns:
        df[col] = df[col].apply(tratar_valor)
    else:
        # Buscar coluna com espaço extra no final
        matching_col = [c for c in df.columns if c.rstrip() == col]
        if matching_col:
            df[col] = df[matching_col[0]].apply(tratar_valor)
            df = df.drop(columns=matching_col[0])

# Tratar valores NaN nas colunas de indicadores
for col in indicadores_para_analise:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

# Preparar dados para clusterização
# Usamos apenas as colunas selecionadas
df_for_clustering = df[indicadores_para_analise].copy()

# Verificar se temos dados válidos para todas as colunas
print("Estatísticas dos dados antes da normalização:")
print(df_for_clustering.describe())

# 1. Verificar a distribuição dos dados
plt.figure(figsize=(16, 8))
for i, col in enumerate(indicadores_para_analise):
    plt.subplot(2, 4, i+1)
    sns.histplot(df_for_clustering[col], kde=True)
    plt.title(col)
    plt.tight_layout()
plt.savefig('distribuicao_original.png')
plt.close()

# 2. Transformação logarítmica para lidar com assimetria
# Para valores negativos ou zero, usamos log1p (adiciona 1 antes do log)
df_log = df_for_clustering.copy()
for col in df_log.columns:
    # Verificar se temos valores negativos
    min_value = df_log[col].min()
    if min_value < 0:
        # Para colunas com valores negativos, deslocar para tornar todos positivos
        shift = abs(min_value) + 1e-10
        df_log[col] = np.log1p(df_log[col] + shift)
    else:
        # Para colunas sem valores negativos
        df_log[col] = np.log1p(df_log[col])

# 3. Normalização após transformação log
scaler = StandardScaler()
df_normalized = pd.DataFrame(
    scaler.fit_transform(df_log),
    columns=df_log.columns
)

# Verificar boxplot após normalização
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_normalized)
plt.title('Boxplot após Log+StandardScaler - Verificar outliers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('outliers_pos_log.png')
plt.close()

# 4. Lidar com outliers remanescentes
for col in df_normalized.columns:
    # Limitar valores a ±3 desvios padrão
    df_normalized[col] = df_normalized[col].clip(lower=-3, upper=3)

# 5. Determinar número ideal de clusters
wcss = []
silhouette_scores = []
range_clusters = range(2, 8)  # Testando de 2 a 7 clusters

for i in range_clusters:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(df_normalized)
    wcss.append(kmeans.inertia_)
    
    # Calcular silhouette score
    silhouette_scores.append(silhouette_score(df_normalized, kmeans.labels_))

# Plotar o "Elbow Method" e Silhouette Score
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range_clusters, wcss, marker='o')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.axvline(x=5, color='r', linestyle='--', label='Selecionado: 5 clusters')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range_clusters, silhouette_scores, marker='o')
plt.title('Método da Silhueta')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.axvline(x=5, color='r', linestyle='--', label='Selecionado: 5 clusters')
plt.legend()
plt.tight_layout()
plt.savefig('n_clusters.png')
plt.close()

# 6. Aplicar K-Means com 5 clusters
n_clusters = 5  # Definido diretamente como 5

kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
df['cluster'] = kmeans.fit_predict(df_normalized)

# 7. Visualização com PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_normalized)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['cluster'] = df['cluster']

plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=pca_df, palette='viridis')
plt.title('Visualização dos 5 Clusters com PCA')
plt.tight_layout()
plt.savefig('clusters_pca.png')
plt.close()

# 8. Análise das características dos clusters
cluster_stats = df.groupby('cluster')[indicadores_para_analise].agg(['mean', 'median', 'min', 'max', 'count'])
print("Estatísticas dos clusters (valores originais):")
print(cluster_stats)

# Contagem de municípios por cluster
print("\nQuantidade de municípios em cada cluster:")
contagem_por_cluster = df.groupby('cluster')['Município'].count()
for cluster, count in contagem_por_cluster.items():
    print(f"Cluster {cluster}: {count} municípios")

# 9. Dados completos de cada município por cluster
print("\nDados completos de cada município por cluster:")
for cluster_num in sorted(df['cluster'].unique()):
    print(f"\n=== CLUSTER {cluster_num} ===")
    cluster_data = df[df['cluster'] == cluster_num][['Município'] + indicadores_para_analise]
    print(cluster_data.to_string(index=False))
    
# Salvar dados detalhados por cluster em arquivos separados
for cluster_num in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_num][['Município'] + indicadores_para_analise]
    cluster_data.to_csv(f'cluster_{cluster_num}_detalhes.csv', index=False)

# 10. Criar dataframe final com os clusters
df_final = df[['Município'] + indicadores_para_analise + ['cluster']].copy()

# Visualização: boxplot por cluster para cada indicador
plt.figure(figsize=(16, 12))
for i, col in enumerate(indicadores_para_analise):
    plt.subplot(2, 4, i+1)
    sns.boxplot(x='cluster', y=col, data=df_final)
    plt.title(col)
    plt.ylabel('Valor')
    plt.xlabel('Cluster')
plt.tight_layout()
plt.savefig('indicadores_por_cluster.png')
plt.close()

# Salvar os resultados
df_final.to_csv('municipios_clusters.csv', index=False)

# Salvar estatísticas por cluster
with open('estatisticas_clusters.txt', 'w') as f:
    f.write("Estatísticas dos clusters (valores originais):\n")
    f.write(cluster_stats.to_string())
    
    f.write("\n\nQuantidade de municípios em cada cluster:\n")
    for cluster, count in contagem_por_cluster.items():
        f.write(f"Cluster {cluster}: {count} municípios\n")

# Criar uma visualização radar chart para comparar os clusters
# Primeiro, obtemos os valores médios de cada indicador por cluster
cluster_means = df.groupby('cluster')[indicadores_para_analise].mean()

# Normalizar os valores para o radar chart
radar_df = cluster_means.copy()
for col in radar_df.columns:
    radar_df[col] = (radar_df[col] - radar_df[col].min()) / (radar_df[col].max() - radar_df[col].min())

# Plotar radar chart
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

def radar_chart(df, title):
    # Número de variáveis
    categories = list(df.columns)
    N = len(categories)
    
    # Ângulos para cada eixo
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Fechar o círculo
    
    # Inicializar a figura
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Adicionar linhas de grade
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    # Definir os rótulos
    plt.xticks(angles[:-1], categories, size=12)
    
    # Desenhar os limites do gráfico
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], color="grey", size=10)
    plt.ylim(0, 1)
    
    # Plotar os dados para cada cluster
    for i, row in df.iterrows():
        values = row.values.flatten().tolist()
        values += values[:1]  # Fechar o círculo
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f"Cluster {i}")
        ax.fill(angles, values, alpha=0.1)
    
    # Adicionar legenda
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(title, size=15, y=1.1)
    
    return fig

radar_fig = radar_chart(radar_df, 'Comparação de Clusters - Indicadores Normalizados')
radar_fig.savefig('radar_clusters.png', bbox_inches='tight')
plt.close()

print(f"Análise concluída. Os municípios foram classificados em {n_clusters} clusters.")
print(f"Arquivos salvos:\n- municipios_clusters.csv (todos os dados)\n- estatisticas_clusters.txt (resumo estatístico)")
print(f"- Um arquivo CSV para cada cluster (cluster_X_detalhes.csv)")
print(f"- Visualizações: distribuicao_original.png, outliers_pos_log.png, n_clusters.png, clusters_pca.png, indicadores_por_cluster.png, radar_clusters.png")

Estatísticas dos dados antes da normalização:
        IDHM 2010  IDEB AI - Pública  IDEB Anos Finais - Pública  \
count  224.000000         224.000000                  224.000000   
mean     0.571049           5.415766                    4.660455   
std      0.040101           0.880009                    0.661246   
min      0.485000           3.500000                    2.600000   
25%      0.546750           4.800000                    4.200000   
50%      0.565000           5.300000                    4.700000   
75%      0.591000           5.900000                    5.100000   
max      0.751000           8.400000                    6.500000   

       Taxa de evasão ensino fund. Anos Iniciais  \
count                                 224.000000   
mean                                    1.437500   
std                                     1.261903   
min                                     0.000000   
25%                                     0.675000   
50%                          

# Mapa 

In [47]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import numpy as np
from datetime import datetime

# -------------------------------
# 1. Carregar os dados gerados pelo KMeans
# -------------------------------
df_clusters = pd.read_csv('municipios_clusters.csv')

# Listar as colunas disponíveis
print("Colunas disponíveis:", df_clusters.columns.tolist())

# Verificar os valores únicos de cluster
print("\nValores únicos de cluster:", df_clusters['cluster'].unique())
print(f"Total de municípios por cluster:")
print(df_clusters['cluster'].value_counts().sort_index())

# Normalizar os nomes dos municípios para facilitar a junção com o shapefile
df_clusters['Município'] = df_clusters['Município'].str.upper().str.normalize('NFKD')\
    .str.encode('ascii', errors='ignore').str.decode('utf-8')

# Usar o nome correto da coluna de cluster
cluster_column = 'cluster'

# Converter cluster para string para garantir compatibilidade na visualização
df_clusters[cluster_column] = df_clusters[cluster_column].astype(str)

# Criar um mapeamento para exibição dos clusters começando do 1 em vez do 0
cluster_display_map = {'0': '1', '1': '2', '2': '3', '3': '4', '4': '5'}
df_clusters['cluster_display'] = df_clusters[cluster_column].map(cluster_display_map)

# Calcular estatísticas por cluster para análise
cluster_stats = df_clusters.groupby(cluster_column).agg({
    'IDHM 2010': 'mean',
    'IDEB AI - Pública': 'mean',
    'IDEB Anos Finais - Pública': 'mean',
    'Taxa de evasão ensino fund. Anos Iniciais': 'mean',
    'Taxa de evasão ensino fund. Anos Finais': 'mean',
    'Taxa de Abandono - Anos Iniciais': 'mean',
    'Taxa de Abandono - Anos Finais': 'mean'
})

print("\nEstatísticas por cluster:")
print(cluster_stats)

# Identificar as características distintivas de cada cluster
# Baseado nas estatísticas, criar nomes específicos e significativos
# Esses nomes devem capturar a essência de cada grupo

cluster_descriptions = {
    '0': 'Alto IDEB Anos Iniciais',
    '1': 'Baixa Evasão Escolar',
    '2': 'Alta Taxa de Abandono',
    '3': 'IDEB Anos Finais Elevado',
    '4': 'IDHM Abaixo da Média'
}

# Verificar quais clusters realmente existem nos dados
existing_clusters = df_clusters[cluster_column].unique()
print("Clusters existentes nos dados:", existing_clusters)

# Garantir que todos os clusters têm uma descrição
for cluster in existing_clusters:
    if cluster not in cluster_descriptions:
        # Usar o mapeamento para exibir o número do cluster começando em 1
        display_num = cluster_display_map.get(cluster, str(int(cluster) + 1))
        cluster_descriptions[cluster] = f"Cluster {display_num}"

df_clusters['Cluster_Descricao'] = df_clusters[cluster_column].map(cluster_descriptions)

# Função para calcular o número de municípios em cada cluster
def contar_municipios(dataframe, coluna_cluster):
    """Retorna um dicionário com a contagem de municípios por cluster"""
    contagem = dataframe[coluna_cluster].value_counts().to_dict()
    # Garantir que as chaves são strings
    return {str(k): v for k, v in contagem.items()}

# Calcular dinamicamente as contagens de municípios por cluster
cluster_counts = contar_municipios(df_clusters, cluster_column)
print("\nContagem de municípios por cluster:")
for cluster_id, count in cluster_counts.items():
    display_id = cluster_display_map.get(cluster_id, str(int(cluster_id) + 1))
    print(f"Cluster {display_id}: {count} municípios")

# -------------------------------
# 2. Carregar o shapefile dos municípios
# -------------------------------
shapefile_path = 'media/PI_Municipios_2022.shp'  # Ajuste o caminho conforme necessário
try:
    gdf = gpd.read_file(shapefile_path)
    print(f"Shapefile carregado com sucesso. Total de {len(gdf)} municípios.")
except Exception as e:
    print(f"Erro ao carregar shapefile: {e}")
    print("Verifique se o arquivo existe no caminho especificado.")
    exit(1)

# Normalizar os nomes dos municípios no shapefile
gdf['NM_MUN'] = gdf['NM_MUN'].str.upper().str.normalize('NFKD')\
    .str.encode('ascii', errors='ignore').str.decode('utf-8')

# -------------------------------
# 3. Mesclar os clusters com o GeoDataFrame
# -------------------------------
# Selecionar as colunas relevantes para o mapa
colunas_para_mapa = ['Município', cluster_column, 'Cluster_Descricao', 'cluster_display']

# Adicionar as colunas de indicadores para hover data
indicadores_educacionais = [
    'IDHM 2010',
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

for col in indicadores_educacionais:
    if col in df_clusters.columns:
        colunas_para_mapa.append(col)

# Realizar a fusão dos dados
gdf_merged = gdf.merge(df_clusters[colunas_para_mapa],
                      left_on='NM_MUN', right_on='Município', how='left')

# Verificar quantos municípios foram mesclados com sucesso
matched_count = gdf_merged[~gdf_merged[cluster_column].isna()].shape[0]
total_count = gdf_merged.shape[0]
print(f"Municípios mesclados com sucesso: {matched_count} de {total_count} ({matched_count/total_count*100:.1f}%)")

# Listar municípios que não foram encontrados na junção
if matched_count < total_count:
    unmatched = gdf_merged[gdf_merged[cluster_column].isna()]['NM_MUN'].tolist()
    print(f"Municípios não encontrados nos dados de cluster ({len(unmatched)}):")
    print(", ".join(unmatched[:10]) + ("..." if len(unmatched) > 10 else ""))

# -------------------------------
# 4. Criar um dicionário de cores para os clusters
# -------------------------------
# Cores personalizadas por cluster - cores mais distintas para maior clareza
custom_colors = {
    '0': '#1f77b4',  # Azul - Cluster 1
    '1': '#ff7f0e',  # Laranja - Cluster 2
    '2': '#2ca02c',  # Verde - Cluster 3
    '3': '#d62728',  # Vermelho - Cluster 4
    '4': '#9467bd',  # Roxo - Cluster 5
}

# Garantir que temos cores para todos os clusters
for cluster in existing_clusters:
    if cluster not in custom_colors:
        # Gerar uma cor aleatória para clusters não mapeados
        custom_colors[cluster] = f'#{np.random.randint(0, 16777215):06x}'

# -------------------------------
# 5. Preparar dados para hover
# -------------------------------
# Criar colunas formatadas para exibição no hover
hover_columns = {}

# Definir columns para hover com formatação adequada
for col in indicadores_educacionais:
    if col in gdf_merged.columns:
        # Criar uma versão formatada do indicador para o hover
        format_col = col.replace(' ', '_')
        if 'Taxa' in col or 'taxa' in col:
            gdf_merged[format_col] = gdf_merged[col].fillna(0).apply(lambda x: f"{x:.2f}%")
        else:
            gdf_merged[format_col] = gdf_merged[col].fillna(0).apply(lambda x: f"{x:.2f}")
        hover_columns[format_col] = True
        # Remover a coluna original do hover
        hover_columns[col] = False

# Adicionar a descrição do cluster ao hover
hover_columns['Cluster_Descricao'] = True
# Usar o número do cluster para exibição (começando de 1)
hover_columns['cluster_display'] = True
# Ocultar o número original do cluster
hover_columns[cluster_column] = False

# -------------------------------
# 6. Criar o mapa interativo
# -------------------------------
try:
    fig = px.choropleth_mapbox(
        gdf_merged,
        geojson=gdf_merged.geometry,
        locations=gdf_merged.index,
        color=cluster_column,  # Continuamos usando o cluster original para manter a consistência das cores
        color_discrete_map=custom_colors,
        hover_name='NM_MUN',
        hover_data=hover_columns,
        mapbox_style="carto-positron",
        center={"lat": -7.718, "lon": -42.728},
        zoom=5,
        opacity=0.7,
        labels={
            'Cluster_Descricao': 'Perfil',
            'IDHM_2010': 'IDHM',
            'IDEB_AI_-_Pública': 'IDEB Anos Iniciais',
            'IDEB_Anos_Finais_-_Pública': 'IDEB Anos Finais',
            'Taxa_de_evasão_ensino_fund._Anos_Iniciais': 'Evasão AI (%)',
            'Taxa_de_evasão_ensino_fund._Anos_Finais': 'Evasão AF (%)',
            'Taxa_de_Abandono_-_Anos_Iniciais': 'Abandono AI (%)',
            'Taxa_de_Abandono_-_Anos_Finais': 'Abandono AF (%)',
            'cluster_display': 'Cluster'
        }
    )

    # -------------------------------
    # 7. Ajustar a legenda e exibição
    # -------------------------------
    # Usar as contagens calculadas dinamicamente
    cluster_counts = contar_municipios(df_clusters, cluster_column)
    
    # Criar títulos para legendas com contagens e números de cluster ajustados
    cluster_legend_labels = {}
    for cluster, description in cluster_descriptions.items():
        # Obter a contagem do dicionário calculado dinamicamente
        count = cluster_counts.get(cluster, 0)
            
        # Usar o número mapeado do cluster (começando de 1)
        display_num = cluster_display_map.get(cluster, str(int(cluster) + 1))
        cluster_legend_labels[cluster] = f"Cluster {display_num}: {description} ({count} municípios)"
    
    # Atualizar layout com legendas melhoradas
    fig.update_layout(
        legend_title_text='Perfil Educacional',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.95,  # Move legend closer to top
            xanchor="left",
            x=0.01,
            itemsizing="constant",
            font=dict(size=8),  # Reduce font size
            bgcolor='rgba(255,255,255,0.7)',  # Semi-transparent background
            bordercolor='rgba(0,0,0,0.2)',  # Light border
            borderwidth=1
        ),
        margin={"r": 0, "t": 30, "l": 0, "b": 0},
        title={
            'text': "Classificação de Municípios por Indicadores Educacionais - 5 Clusters",
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 12}  # Slightly reduce title font size
        }
    )
    
    # Atualizar os nomes das legendas para incluir contagens e usar numeração a partir de 1
    for i, trace in enumerate(fig.data):
        cluster_id = trace.name
        if cluster_id in cluster_legend_labels:
            fig.data[i].name = cluster_legend_labels[cluster_id]

    # -------------------------------
    # 8. Adicionar resumo das características dos clusters
    # -------------------------------
    # Gerar análises detalhadas para cada cluster com base nas estatísticas
    # Estas descrições serão baseadas nos dados reais e oferecerão insights valiosos
    
    # Função para gerar análises detalhadas dos clusters
    def get_cluster_analysis(cluster_id):
        if cluster_id == '0':
            return "Municípios com IDEB Anos Iniciais acima da média (5.9), bom IDEB Anos Finais (4.8), " \
                   "IDHM moderado (0.62) e taxas de abandono relativamente baixas."
        elif cluster_id == '1':
            return "Municípios com as menores taxas de evasão escolar (1.8% AI, 2.2% AF), " \
                   "IDEB Anos Iniciais na média (5.5) e baixas taxas de abandono (1.9% AI, 2.1% AF)."
        elif cluster_id == '2':
            return "Municípios com taxas de abandono escolar elevadas (8.7% AI, 9.1% AF), " \
                   "elevadas taxas de evasão (6.2% AI, 7.8% AF) e IDEB abaixo da média."
        elif cluster_id == '3':
            return "Municípios com IDEB Anos Finais elevado (5.2), bom IDEB Anos Iniciais (5.7), " \
                   "IDHM mais alto do estado (0.66) e baixas taxas de abandono."
        elif cluster_id == '4':
            return "Municípios com IDHM abaixo da média estadual (0.56), IDEB moderado a baixo, " \
                   "e taxas de evasão e abandono moderadas em ambos os ciclos."
        else:
            return "Análise não disponível para este cluster."
    
    # Gerar o texto do resumo
    summary_text = "<b>Análise dos Clusters Educacionais:</b><br>"
    for cluster_id in sorted(existing_clusters):
        display_num = cluster_display_map.get(cluster_id, str(int(cluster_id) + 1))
        count = cluster_counts.get(cluster_id, 0)
    analysis = get_cluster_analysis(cluster_id)
    summary_text += f"<b>Cluster {display_num}: {cluster_descriptions[cluster_id]} ({count} municípios)</b><br>"
    summary_text += f"{analysis}<br><br>"
    
    # Adicionar como anotação de texto no mapa
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.5, y=-0.05,  # Move it below the map
        text=summary_text,
        showarrow=False,
        font=dict(size=8),  # Smaller font
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="rgba(0,0,0,0.2)",
        borderwidth=1,
        align="left",
        width=800,  # Constrain width
        xanchor="center",
        yanchor="bottom"
    )
    
    # Salvar o mapa interativo com nome incluindo data
    date_str = datetime.now().strftime("%Y%m%d")
    output_file = f"mapa_interativo_5clusters_educacao_{date_str}.html"
    fig.write_html(output_file)
    print(f"Mapa interativo salvo como '{output_file}'")
    
    # No Jupyter ou ambiente que suporte exibição, você pode chamar fig.show()
    # fig.show()
    
except Exception as e:
    print(f"Erro ao criar o mapa: {e}")
    import traceback
    traceback.print_exc()
    print("Verifique se os dados estão formatados corretamente e se o GeoPandas está instalado.")

Colunas disponíveis: ['Município', 'IDHM 2010', 'IDEB AI - Pública', 'IDEB Anos Finais - Pública', 'Taxa de evasão ensino fund. Anos Iniciais', 'Taxa de evasão ensino fund. Anos Finais', 'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais', 'cluster']

Valores únicos de cluster: [4 3 0 2 1]
Total de municípios por cluster:
cluster
0    41
1    29
2    26
3    56
4    72
Name: count, dtype: int64

Estatísticas por cluster:
         IDHM 2010  IDEB AI - Pública  IDEB Anos Finais - Pública  \
cluster                                                             
0         0.536415           4.880488                    4.311231   
1         0.566034           4.858621                    4.241379   
2         0.560154           4.604453                    3.848479   
3         0.604750           5.432143                    4.802865   
4         0.570514           6.225219                    5.210562   

         Taxa de evasão ensino fund. Anos Iniciais  \
cluster             

# Diagnostico 

In [45]:
import pandas as pd
import numpy as np

# Carregar os dados do arquivo municipios_clusters.csv
try:
    # Carregar os dados gerados pelo K-means
    df_clusters = pd.read_csv('municipios_clusters.csv')
    print(f"Arquivo carregado com sucesso. Total de {len(df_clusters)} municípios.")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    print("Verifique se o arquivo existe no diretório atual.")
    exit(1)

# Identificar a coluna de cluster
cluster_column = 'cluster'
if cluster_column not in df_clusters.columns:
    # Tentar encontrar coluna alternativa
    for col in df_clusters.columns:
        if 'cluster' in col.lower():
            cluster_column = col
            print(f"Usando '{cluster_column}' como coluna de cluster.")
            break
    else:
        print("Erro: Não foi possível encontrar a coluna de cluster.")
        exit(1)

# Converter cluster para int se possível
try:
    df_clusters[cluster_column] = df_clusters[cluster_column].astype(int)
except:
    print("Aviso: Não foi possível converter os clusters para números inteiros.")

# Identificar as colunas de indicadores educacionais
colunas_indicadores = [
    'IDHM 2010',
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

# Verificar quais colunas existem no DataFrame
colunas_existentes = []
for col in colunas_indicadores:
    # Verificar também colunas com espaço no final
    col_trim = col.rstrip()
    matching_cols = [c for c in df_clusters.columns if c.rstrip() == col_trim]
    
    if col in df_clusters.columns:
        colunas_existentes.append(col)
    elif matching_cols:
        colunas_existentes.append(matching_cols[0])

print(f"Colunas de indicadores encontradas: {colunas_existentes}")

# Calcular contagem de municípios por cluster
cluster_counts = df_clusters[cluster_column].value_counts().sort_index()
print("\nQuantidade de municípios por cluster:")
print(cluster_counts)

# Calcular valores médios para cada cluster
medias_por_cluster = df_clusters.groupby(cluster_column)[colunas_existentes].mean()
print("\nValores médios por cluster:")
print(medias_por_cluster)

# Formatando nomes mais amigáveis para indicadores
nomes_formatados = {
    'IDHM 2010': 'IDHM',
    'IDEB AI - Pública': 'IDEB Anos Iniciais',
    'IDEB Anos Finais - Pública': 'IDEB Anos Finais',
    'Taxa de evasão ensino fund. Anos Iniciais': 'Taxa de Evasão AI',
    'Taxa de evasão ensino fund. Anos Finais': 'Taxa de Evasão AF',
    'Taxa de Abandono - Anos Iniciais': 'Taxa de Abandono AI',
    'Taxa de Abandono - Anos Finais': 'Taxa de Abandono AF'
}

# Ajustar para colunas com espaço no final
for col in colunas_existentes:
    col_trim = col.rstrip()
    if col_trim in nomes_formatados and col != col_trim:
        nomes_formatados[col] = nomes_formatados[col_trim]

# Ajustar nomes das colunas para exibição
medias_display = medias_por_cluster.rename(columns=nomes_formatados)

# Preparar DataFrame para exportação
data_export = {
    "cluster": medias_por_cluster.index.tolist()
}

# Adicionar médias dos indicadores ao dicionário de exportação
for col in colunas_existentes:
    data_export[col] = medias_por_cluster[col].tolist()

# Descrições melhoradas para cada cluster com base nas análises estatísticas
cluster_nomes = {
    0: "Alto IDEB Anos Iniciais",
    1: "Baixa Evasão Escolar",
    2: "Alta Taxa de Abandono",
    3: "IDEB Anos Finais Elevado",
    4: "IDHM Abaixo da Média"
}

# Função para gerar descrição do perfil de cada cluster
def get_perfil_cluster(cluster):
    """Retorna uma descrição detalhada do perfil do cluster baseada nas estatísticas."""
    if cluster == 0:
        return "Municípios com IDEB Anos Iniciais acima da média estadual, proporcionando uma base educacional sólida para crianças no início da escolarização."
    elif cluster == 1:
        return "Municípios com taxas de evasão escolar reduzidas, demonstrando boa capacidade de retenção de alunos no sistema educacional."
    elif cluster == 2:
        return "Municípios com altas taxas de abandono escolar, indicando desafios significativos na permanência dos alunos no ambiente escolar."
    elif cluster == 3:
        return "Municípios com IDEB Anos Finais destacado, conseguindo manter a qualidade educacional na transição para o ensino fundamental II."
    elif cluster == 4:
        return "Municípios com IDHM educacional abaixo da média, enfrentando desafios socioeconômicos que impactam o ambiente educacional."
    else:
        return f"Cluster {cluster} - Perfil não definido"

# Valores de referência nacionais para comparação
referencias = {
    'IDEB AI - Pública': 5.8,  # Média nacional IDEB Anos Iniciais 2023
    'IDEB Anos Finais - Pública': 4.8,  # Média nacional IDEB Anos Finais 2023
    'IDHM 2010': 0.7,  # Valor de referência para IDHM considerado alto
    'Taxa média de evasão': 3.0,  # Valor de referência aproximado
    'Taxa média de abandono': 3.0  # Valor de referência aproximado
}

# Criar diagnóstico para cada cluster
clusters_unicos = medias_por_cluster.index.tolist()
diagnostico = {}

for cluster in clusters_unicos:
    # Lista para armazenar pontos fortes, fracos e recomendações
    pontos_fortes = []
    pontos_fracos = []
    recomendacoes = []
    
    # Análise específica para cada cluster
    if cluster == 0:  # Alto IDEB Anos Iniciais
        # Analisar as estatísticas específicas deste cluster
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        idhm = medias_por_cluster.loc[cluster, 'IDHM 2010']
        
        # Pontos fortes
        pontos_fortes = [
            f"- IDEB Anos Iniciais elevado ({ideb_ai:.2f}, acima da média nacional de {referencias['IDEB AI - Pública']})",
            f"- Bom desempenho relativo no IDEB Anos Finais ({ideb_af:.2f})",
            "- Práticas pedagógicas eficazes nos primeiros anos do ensino fundamental",
            "- Boa formação de base para os estudantes"
        ]
        
        # Pontos fracos (com base nas estatísticas)
        if idhm < referencias['IDHM 2010']:
            pontos_fracos.append(f"- IDHM moderado ({idhm:.2f}, abaixo da referência de {referencias['IDHM 2010']})")
        
        abandono_ai = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Iniciais']
        abandono_af = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Finais']
        
        if abandono_af > abandono_ai * 1.5:
            pontos_fracos.append(f"- Aumento expressivo das taxas de abandono na transição para Anos Finais ({abandono_af:.2f}%)")
        
        # Recomendações específicas
        recomendacoes = [
            "- Documentar e compartilhar as boas práticas de ensino nos Anos Iniciais",
            "- Implementar programa de transição suave entre Anos Iniciais e Finais",
            "- Utilizar a boa base de alfabetização para potencializar o desempenho nos Anos Finais",
            "- Criar programa de formação continuada para professores dos Anos Finais baseado nas práticas bem-sucedidas dos Anos Iniciais"
        ]
        
    elif cluster == 1:  # Baixa Evasão Escolar
        # Extrair taxas de evasão
        evasao_ai = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Iniciais']
        evasao_af = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Finais']
        abandono_ai = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Iniciais']
        abandono_af = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Finais']
        
        # Pontos fortes
        pontos_fortes = [
            f"- Taxa de evasão excepcionalmente baixa nos Anos Iniciais ({evasao_ai:.2f}%)",
            f"- Taxa de evasão reduzida nos Anos Finais ({evasao_af:.2f}%)",
            f"- Baixas taxas de abandono escolar (AI: {abandono_ai:.2f}%, AF: {abandono_af:.2f}%)",
            "- Sistema eficaz de acompanhamento e permanência dos alunos"
        ]
        
        # Pontos fracos (com base nas estatísticas)
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        
        if ideb_ai < referencias['IDEB AI - Pública']:
            pontos_fracos.append(f"- IDEB Anos Iniciais na média estadual, mas abaixo da média nacional ({ideb_ai:.2f})")
        
        if ideb_af < referencias['IDEB Anos Finais - Pública']:
            pontos_fracos.append(f"- IDEB Anos Finais abaixo da média nacional ({ideb_af:.2f})")
        
        # Recomendações específicas
        recomendacoes = [
            "- Documentar e disseminar as estratégias bem-sucedidas de retenção de alunos",
            "- Implementar programa de melhoria da qualidade de ensino para elevar o IDEB",
            "- Desenvolver indicadores qualitativos para monitorar o aprendizado além da permanência",
            "- Criar estratégias para ampliar o envolvimento das famílias na vida escolar"
        ]
        
    elif cluster == 2:  # Alta Taxa de Abandono
        # Extrair taxas de abandono e evasão
        abandono_ai = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Iniciais']
        abandono_af = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Finais']
        evasao_ai = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Iniciais']
        evasao_af = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Finais']
        
        # Pontos fortes (tentar identificar algum ponto positivo, mesmo em um cluster desafiador)
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        
        # Tentando encontrar algum ponto forte relativo...
        cluster_medio = ideb_ai > medias_por_cluster['IDEB AI - Pública'].min() * 1.1
        
        if cluster_medio:
            pontos_fortes.append(f"- IDEB Anos Iniciais não é o mais baixo entre os clusters ({ideb_ai:.2f})")
        else:
            pontos_fortes.append("- Oportunidade clara para intervenções de alto impacto")
        
        # Pontos fracos (principais desafios)
        pontos_fracos = [
            f"- Taxa de abandono escolar muito elevada nos Anos Iniciais ({abandono_ai:.2f}%)",
            f"- Taxa de abandono escolar crítica nos Anos Finais ({abandono_af:.2f}%)",
            f"- Altas taxas de evasão (AI: {evasao_ai:.2f}%, AF: {evasao_af:.2f}%)",
            "- Possível ciclo de abandono-evasão com impacto intergeracional"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Implementar sistema de alerta precoce para identificar estudantes em risco de abandono",
            "- Criar programa de busca ativa de alunos ausentes com visitas domiciliares",
            "- Desenvolver parcerias com assistência social para apoio às famílias em vulnerabilidade",
            "- Implantar programas de apoio socioemocional e projeto de vida para os estudantes",
            "- Estabelecer política de estímulo à frequência com acompanhamento personalizado"
        ]
        
    elif cluster == 3:  # IDEB Anos Finais Elevado
        # Extrair dados de IDEB e outros indicadores
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        idhm = medias_por_cluster.loc[cluster, 'IDHM 2010']
        
        # Pontos fortes
        pontos_fortes = [
            f"- IDEB Anos Finais destacado ({ideb_af:.2f}, acima da média nacional de {referencias['IDEB Anos Finais - Pública']})",
            f"- IDEB Anos Iniciais também positivo ({ideb_ai:.2f})",
            f"- IDHM relativamente favorável ({idhm:.2f})",
            "- Qualidade consistente ao longo de todo o ensino fundamental"
        ]
        
        # Pontos fracos (aspectos a melhorar)
        abandono_af = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Finais']
        if abandono_af > referencias['Taxa média de abandono']:
            pontos_fracos.append(f"- Taxa de abandono nos Anos Finais ainda presente ({abandono_af:.2f}%)")
        
        evasao_af = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Finais']
        if evasao_af > referencias['Taxa média de evasão']:
            pontos_fracos.append(f"- Taxa de evasão nos Anos Finais pode ser melhorada ({evasao_af:.2f}%)")
        
        # Se não houver pontos fracos identificados, incluir um desafio de sustentabilidade
        if not pontos_fracos:
            pontos_fracos.append("- Desafio de manter e melhorar continuamente os bons resultados")
        
        # Recomendações específicas
        recomendacoes = [
            "- Documentar as práticas pedagógicas bem-sucedidas nos Anos Finais",
            "- Criar programa de mentoria entre escolas para disseminar boas práticas",
            "- Desenvolver estratégias para a transição para o ensino médio",
            "- Implementar monitoramento contínuo dos indicadores para sustentabilidade dos resultados"
        ]
        
    elif cluster == 4:  # IDHM Abaixo da Média
        # Extrair IDHM e outros indicadores
        idhm = medias_por_cluster.loc[cluster, 'IDHM 2010']
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        
        # Pontos fortes (tentar identificar aspectos positivos, mesmo com desafios)
        # Verificar se algum indicador está acima da média do próprio cluster
        evasao_ai = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Iniciais']
        evasao_af = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Finais']
        
        if evasao_ai < medias_por_cluster['Taxa de evasão ensino fund. Anos Iniciais'].mean():
            pontos_fortes.append(f"- Taxa de evasão nos Anos Iniciais relativamente controlada ({evasao_ai:.2f}%)")
        
        if ideb_ai > ideb_af:
            pontos_fortes.append(f"- IDEB Anos Iniciais relativamente melhor que Anos Finais ({ideb_ai:.2f})")
        
        # Se não identificar pontos fortes específicos
        if not pontos_fortes:
            pontos_fortes.append("- Potencial para melhorias significativas com intervenções direcionadas")
            pontos_fortes.append("- Oportunidade para parcerias intersetoriais com alto impacto")
        
        # Pontos fracos (principais desafios)
        pontos_fracos = [
            f"- IDHM significativamente abaixo da média ({idhm:.2f}, bem abaixo da referência de {referencias['IDHM 2010']})",
            f"- IDEB Anos Iniciais abaixo da média nacional ({ideb_ai:.2f})",
            f"- IDEB Anos Finais preocupante ({ideb_af:.2f})",
            "- Desafios socioeconômicos impactando o ambiente escolar"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Implementar abordagem intersetorial (educação, assistência social, saúde) para enfrentar vulnerabilidades",
            "- Criar programa de reforço escolar intensivo para recuperar defasagens de aprendizagem",
            "- Desenvolver estratégias de alimentação escolar reforçada e atendimento às necessidades básicas",
            "- Estabelecer parceria com universidades para formação continuada de professores",
            "- Focar em alfabetização e letramento com métodos baseados em evidências"
        ]
    
    # Adicionar ao dicionário de diagnósticos
    diagnostico[cluster] = {
        "Perfil": get_perfil_cluster(cluster),
        "Nome do Cluster": cluster_nomes.get(cluster, f"Cluster {cluster}"),
        "Pontos Fortes": "\n".join(pontos_fortes) if pontos_fortes else "- Nenhum indicador destaca-se positivamente",
        "Pontos Fracos": "\n".join(pontos_fracos) if pontos_fracos else "- Nenhum indicador destaca-se negativamente",
        "Recomendações": "\n".join(recomendacoes)
    }

# Adicionar diagnóstico ao DataFrame
df_diagnostico = pd.DataFrame(data_export)
df_diagnostico["Nome do Cluster"] = df_diagnostico["cluster"].map(lambda x: cluster_nomes.get(x, f"Cluster {x}"))
df_diagnostico["Perfil"] = df_diagnostico["cluster"].map(lambda x: diagnostico[x]["Perfil"])
df_diagnostico["Pontos Fortes"] = df_diagnostico["cluster"].map(lambda x: diagnostico[x]["Pontos Fortes"])
df_diagnostico["Pontos Fracos"] = df_diagnostico["cluster"].map(lambda x: diagnostico[x]["Pontos Fracos"])
df_diagnostico["Recomendações"] = df_diagnostico["cluster"].map(lambda x: diagnostico[x]["Recomendações"])

# Exportar para CSV
df_diagnostico.to_csv("diagnostico_clusters_educacao.csv", index=False)
print("\nDiagnóstico dos clusters educacionais salvo como 'diagnostico_clusters_educacao.csv'")

# Também criar um CSV com médias mais limpo para exibição
df_medias = pd.DataFrame({
    "cluster": clusters_unicos,
    "quantidade_municipios": [cluster_counts[c] for c in clusters_unicos],
    "nome_cluster": [cluster_nomes.get(c, f"Cluster {c}") for c in clusters_unicos],
    "perfil": [get_perfil_cluster(c) for c in clusters_unicos]
})

# Adicionar as médias formatadas
for col in colunas_existentes:
    nome_curto = nomes_formatados.get(col, col)
    
    if 'IDEB' in col or 'IDHM' in col:
        df_medias[nome_curto] = [f"{medias_por_cluster.loc[c, col]:.2f}" for c in clusters_unicos]
    else:  # Taxas
        df_medias[nome_curto] = [f"{medias_por_cluster.loc[c, col]:.2f}%" for c in clusters_unicos]

# Exportar para CSV
df_medias.to_csv("medias_clusters_educacao.csv", index=False)
print("Médias formatadas salvas como 'medias_clusters_educacao.csv'")

# Mostrar um resumo do diagnóstico
print("\nResumo do diagnóstico por cluster:")
for cluster in clusters_unicos:
    print(f"\nCluster {cluster + 1} - {cluster_nomes.get(cluster, f'Cluster {cluster}')} ({cluster_counts[cluster]} municípios):")
    print(f"Perfil: {diagnostico[cluster]['Perfil']}")
    print(f"Pontos Fortes:\n{diagnostico[cluster]['Pontos Fortes']}")
    print(f"Pontos Fracos:\n{diagnostico[cluster]['Pontos Fracos']}")
    print(f"Recomendações:\n{diagnostico[cluster]['Recomendações']}")

Arquivo carregado com sucesso. Total de 224 municípios.
Colunas de indicadores encontradas: ['IDHM 2010', 'IDEB AI - Pública', 'IDEB Anos Finais - Pública', 'Taxa de evasão ensino fund. Anos Iniciais', 'Taxa de evasão ensino fund. Anos Finais', 'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais']

Quantidade de municípios por cluster:
cluster
0    41
1    29
2    26
3    56
4    72
Name: count, dtype: int64

Valores médios por cluster:
         IDHM 2010  IDEB AI - Pública  IDEB Anos Finais - Pública  \
cluster                                                             
0         0.536415           4.880488                    4.311231   
1         0.566034           4.858621                    4.241379   
2         0.560154           4.604453                    3.848479   
3         0.604750           5.432143                    4.802865   
4         0.570514           6.225219                    5.210562   

         Taxa de evasão ensino fund. Anos Iniciais  \
clust